# Experiments

This file allows one to reproduce the experiments with the dataset contained in the folder "dataset" within the repository. This notebook creates a folder named "output_dataset" which contains a csv with the results and the corresponding barcodes in dimensions 0 and 1. Please note that the experiments will need some hours to finish.

The experiments employ the zigzag persistence to compute the number of connected components (0-homology) and the number of holes (1-homology) during the frames of different videos. The components and holes are filtered automatically: only those components that last for more than 20 frames are displayed in the zigzag barcodes.

The ground truth file (ground_truth.txt) contains the real number of connected components and holes (appearing in at least 20 frames) of the videos. This file will be used to compare the results. 

This experiments have been carried out in a laptop with an Intel Core i7-4810MQ 3.8GHz (4 cores, 8 threads) and 24GB RAM, running Ubuntu 20.04.

In [1]:
import dionysus as d
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time
import csv
from pathlib import Path
from natsort import natsorted

In [2]:
aux_zz = []
all_generator_list = []
f = []

def vertexNumber(row,colum,numCols):
    return row*numCols+colum

def addSimplexToSet (s, l):
    l.add(tuple(s))

def binaryImageToSimplicialComplexList(img):
    rowNum = img.shape[0]
    colNum = img.shape[1]
    simplices0 = set()
    simplices1 = set()
    simplices2 = set()
    for j in range(0,rowNum):
        for i in range(0,colNum):
            if (img[j,i]==255):
                v = vertexNumber(j,i,colNum)
                addSimplexToSet([v],simplices0)
                if (j < rowNum-1) and (img[j+1,i] == 255):
                    v10 = vertexNumber(j+1,i,colNum)
#                     addSimplexToSet([v10],simplices0)
                    addSimplexToSet([v,v10],simplices1)
                if (i < colNum-1) and (img[j,i+1] == 255):
                    v01 = vertexNumber(j,i+1,colNum)
#                     addSimplexToSet([v01],simplices0)
                    addSimplexToSet([v,v01],simplices1)                      
                if (i < colNum-1) and (j < rowNum-1) and (img[j+1,i+1] == 255) and (img[j+1,i] == 0) and (img[j,i+1] == 0):
                    v11 = vertexNumber(j+1,i+1,colNum)
#                     addSimplexToSet([v11],simplices0)
                    addSimplexToSet([v,v11],simplices1)
                if (i > 0) and (j < rowNum-1) and (img[j+1,i-1] == 255) and (img[j,i-1] == 0) and (img[j+1,i] == 0):
                    v1_1 = vertexNumber(j+1,i-1,colNum)
#                     addSimplexToSet([v1_1],simplices0)
                    addSimplexToSet([v,v1_1],simplices1)
                if (i < colNum-1) and (j < rowNum-1) and (img[j+1,i+1] == 255) and (img[j+1,i] == 255) and (img[j,i+1] == 255):
                    v11 = vertexNumber(j+1,i+1,colNum)
#                     addSimplexToSet([v11],simplices0)
                    addSimplexToSet([v,v11],simplices1)
#                     addSimplexToSet([v01,v11],simplices1)
#                     addSimplexToSet([v10,v11],simplices1)
                    addSimplexToSet([v,v01,v11],simplices2)
                    addSimplexToSet([v,v10,v11],simplices2)
    return simplices0,simplices1,simplices2

def binaryImageToSimplicialComplex(img):
    simplices0,simplices1,simplices2 = binaryImageToSimplicialComplexList(img)
    simplices0 = sorted(simplices0, key=lambda x: (x[0]))
    for i in range(len(simplices0)):
        simplices0[i] = list(simplices0[i])
    simplices1 = sorted(simplices1, key=lambda x: (x[0],x[1]))
    for i in range(len(simplices1)):
        simplices1[i] = list(simplices1[i])
    simplices2 = sorted(simplices2, key=lambda x: (x[0],x[1],x[2])) 
    for i in range(len(simplices2)):
        simplices2[i] = list(simplices2[i])
    return simplices0+simplices1+simplices2

def simplicesListToZigzagFiltration(slist,n):
    total_simplices_l = []
    total_simplices_set = set()
    indices_l = []
    previous_simplices_l = []
    previous_simplices_set = set()    
    slistaux = []
    aux = []
    
    for sc in slist:
        if n==0:
            aux = sorted(sc, key=lambda x: (x[0]))
        elif n==1:
            aux = sorted(sc, key=lambda x: (x[0],x[1]))
        elif n==2:
            aux = sorted(sc, key=lambda x: (x[0],x[1],x[2]))
        elif n==3:
            aux = sorted(sc, key=lambda x: (x[0],x[1],x[2],x[3]))
    
        for i in range(len(aux)):
            aux[i] = list(aux[i])
        slistaux.append(aux)        
    
    for i,sc in enumerate(slist):
        for s in slistaux[i]:
            if (tuple(s) in total_simplices_set) :
                if not (tuple(s) in previous_simplices_set):
                    index = total_simplices_l.index(s)
                    indices_l[index].append(i)          
            else:
                total_simplices_l.append(s)
                total_simplices_set.add(tuple(s))
                indices_l.append([i])
        for s in previous_simplices_l:
            if not (tuple(s) in sc):
                index = total_simplices_l.index(s)
                indices_l[index].append(i)
        previous_simplices_set = sc
        previous_simplices_l = slistaux[i]   
        
    return total_simplices_l,indices_l
       
def imageListToSimplicialComplexList (imgList):
    
    l0 = []
    l1 = []
    l2 = []
    l3 = []
    for img in imgList:
        sc0,sc1,sc2 = binaryImageToSimplicialComplexList(img)
        l0.append(sc0)
        l1.append(sc1)
        l2.append(sc2)
        l3.append(set())
        
    for r,img in enumerate(imgList):        
        rowNum = img.shape[0]
        colNum = img.shape[1]
        for j in range(0,rowNum):
            for i in range(0,colNum):
                isDiag = 0
                isCorner = 0
                if (img[j,i]==255):
                    if (i < colNum-1) and (j < rowNum-1) and (img[j+1,i+1]==255) and (img[j,i+1]==0) and (img[j+1,i]==0):
                        isDiag = 1
                        diagEdge = [vertexNumber(j,i,colNum),vertexNumber(j+1,i+1,colNum)]
                    elif (i > 0) and (j < rowNum-1) and (img[j+1,i-1] == 255) and (img[j,i-1] == 0) and (img[j+1,i] == 0):
                        isDiag = -1
                        diagEdge = [vertexNumber(j,i,colNum),vertexNumber(j+1,i-1,colNum)]
                    elif (i < colNum-1) and (j < rowNum-1) and (img[j+1,i] == 255) and (img[j,i+1] == 255) and (img[j+1,i+1] == 0):
                        isCorner = 1
                        
                    elif (i > 0) and (j < rowNum-1) and (img[j,i-1] == 255) and (img[j+1,i] == 255) and (img[j+1,i-1] == 0):
                        isCorner = -1
                        
                    if isDiag !=0 or isCorner!=0:
                        imgListAux = []
                        if (r< len(imgList)-1):
                            imgListAux.append([r+1,imgList[r+1]])
                        if (r > 0):
                            imgListAux.append([r-1,imgList[r-1]])
                        for k,img1 in imgListAux:
                            if isDiag==1 and (img1[j,i]==255) and (img1[j+1,i+1]==255) and (img1[j,i+1]==255) and (img1[j+1,i]== 0):
                                addSimplexToSet(diagEdge,l1[k])
                                addSimplexToSet([diagEdge[0],diagEdge[0]+1,diagEdge[1]],l2[k])
                                
                            elif isDiag==1 and (img1[j,i]==255) and (img1[j+1,i+1]==255) and (img1[j,i+1]==0) and (img1[j+1,i]== 255):   
                                addSimplexToSet(diagEdge,l1[k])
                                addSimplexToSet([diagEdge[0],diagEdge[1]-1,diagEdge[1]],l2[k])
                            
                            elif isDiag==-1 and (img1[j,i]==255) and (img1[j+1,i-1]==255) and (img1[j,i-1]==255) and (img1[j+1,i]== 0):
                                addSimplexToSet(diagEdge,l1[k])
                                addSimplexToSet([diagEdge[0]-1,diagEdge[0],diagEdge[1]],l2[k])
                               
                            elif isDiag==-1 and (img1[j,i]==255) and (img1[j+1,i-1]==255) and (img1[j,i-1]==0) and (img1[j+1,i]== 255):
                                addSimplexToSet(diagEdge,l1[k])
                                addSimplexToSet([diagEdge[0],diagEdge[1],diagEdge[1]+1],l2[k])

                            elif isDiag==-1 and (img1[j,i]==255) and (img1[j+1,i-1]==255) and (img1[j,i-1]==255) and (img1[j+1,i]== 255):
                                addSimplexToSet(diagEdge,l1[k])
                                addSimplexToSet([diagEdge[0]-1,diagEdge[0],diagEdge[1]],l2[k])
                                addSimplexToSet([diagEdge[0],diagEdge[1],diagEdge[1]+1],l2[k])
                                addSimplexToSet([diagEdge[0]-1,diagEdge[0],diagEdge[1],diagEdge[1]+1],l3[k])
                                
                            elif isCorner ==1 and img1[j,i]==0 and img1[j,i+1]==255 and img1[j+1,i]==255 and img1[j+1,i+1]==255:
                                diagEdge = [vertexNumber(j,i+1,colNum),vertexNumber(j+1,i,colNum)]
                                addSimplexToSet([diagEdge[0]-1],l0[k])
                                addSimplexToSet(diagEdge,l1[k])
                                addSimplexToSet([diagEdge[0]-1,diagEdge[0]],l1[k])
                                addSimplexToSet([diagEdge[0]-1,diagEdge[1]],l1[k])
                                addSimplexToSet([diagEdge[0]-1,diagEdge[0],diagEdge[1]],l2[k])
                                addSimplexToSet([diagEdge[0],diagEdge[1],diagEdge[1]+1],l2[k])
                                addSimplexToSet([diagEdge[1]+1],l0[r])
                                addSimplexToSet(diagEdge,l1[r])
                                addSimplexToSet([diagEdge[0],diagEdge[1]+1],l1[r])
                                addSimplexToSet([diagEdge[1],diagEdge[1]+1],l1[r])
                                addSimplexToSet([diagEdge[0]-1,diagEdge[0],diagEdge[1]],l2[r])
                                addSimplexToSet([diagEdge[0],diagEdge[1],diagEdge[1]+1],l2[r])
                                
                            elif isCorner ==-1 and img1[j,i]==0 and img1[j,i-1]==255 and img1[j+1,i]==255 and img1[j+1,i-1]==255:
                                diagEdge = [vertexNumber(j,i-1,colNum),vertexNumber(j+1,i,colNum)]
                                addSimplexToSet([diagEdge[0]+1],l0[k])
                                addSimplexToSet(diagEdge,l1[k])
                                addSimplexToSet([diagEdge[0],diagEdge[0]+1],l1[k])
                                addSimplexToSet([diagEdge[0]+1,diagEdge[1]],l1[k])

                                addSimplexToSet([diagEdge[0],diagEdge[0]+1,diagEdge[1]],l2[k])
                                addSimplexToSet([diagEdge[0],diagEdge[1]-1,diagEdge[1]],l2[k])
                                addSimplexToSet([diagEdge[1]-1],l0[r])
                                addSimplexToSet(diagEdge,l1[r])
                                addSimplexToSet([diagEdge[0],diagEdge[1]-1],l1[r])
                                addSimplexToSet([diagEdge[1]-1,diagEdge[1]],l1[r])
                                
                                addSimplexToSet([diagEdge[0],diagEdge[0]+1,diagEdge[1]],l2[r])
                                addSimplexToSet([diagEdge[0],diagEdge[1]-1,diagEdge[1]],l2[r])
    return l0,l1,l2,l3

def imageListToZigzagFiltration(imageList):
                                
    l0,l1,l2,l3 = imageListToSimplicialComplexList(imageList)                     
    zz_list0,zz_indices0 = simplicesListToZigzagFiltration(l0,0)
    zz_list1,zz_indices1 = simplicesListToZigzagFiltration(l1,1)
    zz_list2,zz_indices2 = simplicesListToZigzagFiltration(l2,2)
    zz_list3,zz_indices3 = simplicesListToZigzagFiltration(l3,3)
    
    simplices = zz_list0+zz_list1+zz_list2+zz_list3
    indices = zz_indices0+zz_indices1+zz_indices2+zz_indices3
    return simplices,indices

def detail(i,t,d,zz,cells):
    global aux_zz
    global all_generator_list, f
    
    aux_zz = []  
    for z in zz:
        aux_zz.append(z)        
    if len(aux_zz)!=0:
        aux_zz_last = aux_zz[len(aux_zz)-1]
    else:
        aux_zz_last = []
    z_list = []
    for x in aux_zz_last:
        z_list.append([x.element, f[cells[x.index]]])
    all_generator_list.append(z_list)    
    
def scListZigzagHomology(times):
    global aux_zz
    global all_generator_list, f
                                                    
    aux_zz = []
    all_generator_list = []

    zz, dgms, cells = d.zigzag_homology_persistence(f, times  , callback = detail)
    return dgms, all_generator_list     
        
def imageListZigzagHomology (imageList):
    global f
    
    simplices,times = imageListToZigzagFiltration(imageList)
    f = d.Filtration(simplices)
 
    return scListZigzagHomology(times)

def printGenerator (glist):
    return (' + '.join("%d * %s" % (pair[0], [v for v in pair[1]]) for pair in glist))


def plot_zigzag_bars(dgm, dimension, times, generator_list, interval_l=1, gen_l1=1, gen_l2=0, printGenerators=True,
                     order='birth', ax=None, show= True, **bar_style):
    """
    Plot the barcode.
    Arguments:
        dgm (Diagram): See for example `init_diagrams`.
    Keyword Arguments:
        order (str): How to sort the bars, either 'death' or 'birth'
                     (Default: 'birth')
        show (bool): Display the plot. (Default: False)
        ax (AxesSubplot): Axes that should be used for plotting (Default: None)
        **bar_style: Arguments passed to `ax.plot` for style of the bars.
                     (Defaults: color='b')
    """

    bar_kwargs = {'color': 'b'}
    bar_kwargs.update(bar_style)

    if order == 'death':
        generator = enumerate(sorted(dgm, key=lambda p: p.death))
    else:
        generator = enumerate(dgm)

    if ax is None:
        ax = plt.axes()

    maxi = max(times, key=max)
    maxi = maxi[len(maxi) - 1]

    ax.set_xlim(-0.5, maxi + 1.5)

    result_generators = []

    i = 0
    for j, p in generator:
        if p.death == float('inf'):
            de = maxi + 1
        else:
            de = p.death
        if de - p.birth >= interval_l:
            g = generator_list[p.data]
            if (gen_l1 <= len(g)) and (gen_l2 == 0 or len(g) <= gen_l2):
                if p.death == float('inf'):
                    ax.plot([p.birth, maxi + 1], [i, i], **bar_kwargs)
                    plt.scatter([p.birth], [i], color="blue", edgecolor="blue")
                    result_generators.append(printGenerator(generator_list[p.data]))
                    if printGenerators:
                        plt.text(p.birth, i + 0.2, printGenerator(generator_list[p.data]))
                else:
                    ax.plot([p.birth, p.death - 0.05], [i, i], **bar_kwargs)
                    plt.scatter([p.birth], [i], color="blue", edgecolor="blue")
                    plt.scatter([p.death], [i], color="white", edgecolor="blue")
                    result_generators.append(printGenerator(generator_list[p.data]))
                    if printGenerators:
                        plt.text(p.birth, i + 0.2, printGenerator(generator_list[p.data]))
                i = i + 1

    plt.text(0, i - 0.2, "Barcode of dimension " + str(dimension))
    ax.set_ylim(-1, i)
    plt.yticks(range(0,i))
    plt.xticks(range(0,maxi+2))
    labels = [i for i in range(0,maxi+2)]
    labels[maxi+1]="$+\infty$"
    ax.set_xticklabels(labels, fontsize=12)
    ax.get_xticklabels()[-1].set_fontsize(16)
    ax.set_xticklabels(labels)

    if show:
        plt.show()
        
    return len (result_generators)

def imageListZigzagPlotBar (imageList,dimensions=[0,1],interval_l=1, gen_l1=1, gen_l2=0,printGenerators=True):
    global f

    simplices,times = imageListToZigzagFiltration(imageList)
    f = d.Filtration(simplices)
    dgms, generator_list = scListZigzagHomology(times)
    
    for i in dimensions:
        dgm = dgms[i]
        plot_zigzag_bars(dgm, i, times, generator_list, interval_l, gen_l1, gen_l2,printGenerators)

In [3]:
def binaryImageToCubicalComplexTriangulationList(img):
    rowNum = img.shape[0]
    colNum = img.shape[1]
    simplices0 = set()
    simplices1 = set()
    simplices2 = set()
    for j in range(0,rowNum):
        for i in range(0,colNum):
            if (img[j,i]==255):
                v = vertexNumber(j,i,colNum+1)
                v10 = vertexNumber(j+1,i,colNum+1)
                v01 = vertexNumber(j,i+1,colNum+1)
                v11 = vertexNumber(j+1,i+1,colNum+1)
                addSimplexToSet([v],simplices0)
                addSimplexToSet([v10],simplices0)
                addSimplexToSet([v01],simplices0)
                addSimplexToSet([v11],simplices0)
                
                addSimplexToSet([v,v10],simplices1)          
                addSimplexToSet([v,v01],simplices1)
                addSimplexToSet([v01,v10],simplices1)              
                
                addSimplexToSet([v01,v11],simplices1)
                addSimplexToSet([v10,v11],simplices1)
                addSimplexToSet([v,v01,v10],simplices2)
                addSimplexToSet([v01,v10,v11],simplices2) 
          
    return simplices0,simplices1,simplices2

def binaryImageToCubicalComplexTriangulation(img):
    simplices0,simplices1,simplices2 = binaryImageToCubicalComplexTriangulationList(img)
    simplices0 = sorted(simplices0, key=lambda x: (x[0]))
    for i in range(len(simplices0)):
        simplices0[i] = list(simplices0[i])
    simplices1 = sorted(simplices1, key=lambda x: (x[0],x[1]))
    for i in range(len(simplices1)):
        simplices1[i] = list(simplices1[i])
    simplices2 = sorted(simplices2, key=lambda x: (x[0],x[1],x[2])) 
    for i in range(len(simplices2)):
        simplices2[i] = list(simplices2[i])
    return simplices0+simplices1+simplices2


def imageListCubicalZigzagHomology (imageList):
    global f
    
    l0 = []
    l1 = []
    l2 = []
    for img in imageList:
        sc0,sc1,sc2 = binaryImageToCubicalComplexTriangulationList(img)
        l0.append(sc0)
        l1.append(sc1)
        l2.append(sc2)        
    
    zz_list0,zz_indices0 = simplicesListToZigzagFiltration(l0,0)
    zz_list1,zz_indices1 = simplicesListToZigzagFiltration(l1,1)
    zz_list2,zz_indices2 = simplicesListToZigzagFiltration(l2,2)
    f = d.Filtration(zz_list0+zz_list1+zz_list2)
    times = zz_indices0 + zz_indices1 + zz_indices2   
    
    return scListZigzagHomology(times)

def imageListCubicalZigzagPlotBar (imageList,dimensions=[0,1],interval_l=1, gen_l1=1, gen_l2=0,printGenerators=True):
    global f
    
    l0 = []
    l1 = []
    l2 = []
    for img in imageList:
        sc0,sc1,sc2 = binaryImageToCubicalComplexTriangulationList(img)
        l0.append(sc0)
        l1.append(sc1)
        l2.append(sc2)        
    
    zz_list0,zz_indices0 = simplicesListToZigzagFiltration(l0,0)
    zz_list1,zz_indices1 = simplicesListToZigzagFiltration(l1,1)
    zz_list2,zz_indices2 = simplicesListToZigzagFiltration(l2,2)
    f = d.Filtration(zz_list0+zz_list1+zz_list2)
    times = zz_indices0 + zz_indices1 + zz_indices2 
    dgms, generator_list = scListZigzagHomology(times)
    
    for i in dimensions:
        dgm = dgms[i]
        plot_zigzag_bars(dgm,i, times, generator_list, interval_l, gen_l1, gen_l2,printGenerators)
        plt.show()

In [4]:
def binaryImageToVietorisRipsComplexList(img):
    rowNum = img.shape[0]
    colNum = img.shape[1]
    simplices0 = []
    simplices1 =[]
    simplices2 = []
    simplices3 = []
    points = []
    for j in range(0,rowNum):
        for i in range(0,colNum):
            if (img[j,i]==255):
                points.append([j,i])
                
    points = np.array(points,dtype=float)
                
    f = d.fill_rips(points, 5, 1.43)
    
    for si in f:
        s = [v for v in si]
        if (len(s)==1):
            simplices0.append(s)
        elif (len(s)==2):
            simplices1.append(s)
        elif (len(s)==3):
            simplices2.append(s)
        elif (len(s)==4):
            simplices3.append(s)
 
    simplices0 = sorted(simplices0, key=lambda x: (x[0]))
    simplices1 = sorted(simplices1, key=lambda x: (x[0],x[1]))
    simplices2 = sorted(simplices2, key=lambda x: (x[0],x[1],x[2]))
    simplices3 = sorted(simplices3, key=lambda x: (x[0],x[1],x[2],x[3]))
          
    return simplices0,simplices1,simplices2,simplices3

def binaryImageToVietorisRipsComplex(img):
    simplices0,simplices1,simplices2,simplices3 = binaryImageToVietorisRipsComplexList(img)
    return simplices0+simplices1+simplices2+simplices3

In [5]:
def imageListSizeAndTimeComparison (imageFolder,barcodeFigName,dimensions=[0,1],interval_l=1, gen_l1=1, gen_l2=0,printGenerators=False):
    global f

    imageList = []
    for file_path in natsorted(os.listdir(imageFolder)):
        if os.path.isfile(os.path.join(imageFolder, file_path)):
            img = cv2.imread(os.path.join(imageFolder, file_path),0)
            if not img is None:                
                imageList.append(img)
 
    results = []    
    st = time.time()
    l0,l1,l2,l3 = imageListToSimplicialComplexList(imageList)
    et = time.time()
    complex_time = et - st   
    print("Small complex time:", complex_time)
    st = time.time()
    zz_list0,zz_indices0 = simplicesListToZigzagFiltration(l0,0)
    zz_list1,zz_indices1 = simplicesListToZigzagFiltration(l1,1)
    zz_list2,zz_indices2 = simplicesListToZigzagFiltration(l2,2)
    zz_list3,zz_indices3 = simplicesListToZigzagFiltration(l3,3)    
    simplices = zz_list0+zz_list1+zz_list2+zz_list3
    times = zz_indices0+zz_indices1+zz_indices2+zz_indices3
    f = d.Filtration(simplices)
    dgms, generator_list = scListZigzagHomology(times)
    et = time.time()
    zigzag_time = et - st
    print("Small zigzag time:", zigzag_time)
    for i in dimensions:
        dgm = dgms[i]
        barsNum = plot_zigzag_bars(dgm, i, times, generator_list, interval_l, gen_l1, gen_l2,printGenerators,show=False)
        plt.savefig(barcodeFigName+"_dim_"+str(i)+".png")
        plt.clf()
        results.append(barsNum)
        
    sc0,sc1,sc2 = l0[0],l1[0],l2[0]        
    results.append(len(sc0))
    results.append(len(sc1))
    results.append(len(sc2))
    results.append(complex_time)
    results.append(zigzag_time)

    st = time.time()
    l0 = []
    l1 = []
    l2 = []
    for img in imageList:
        sc0,sc1,sc2 = binaryImageToCubicalComplexTriangulationList(img)
        l0.append(sc0)
        l1.append(sc1)
        l2.append(sc2)        
    et = time.time()
    complex_time = et - st   
    print("Cubical complex time:", complex_time)
    st = time.time()
    zz_list0,zz_indices0 = simplicesListToZigzagFiltration(l0,0)
    zz_list1,zz_indices1 = simplicesListToZigzagFiltration(l1,1)
    zz_list2,zz_indices2 = simplicesListToZigzagFiltration(l2,2)
    f = d.Filtration(zz_list0+zz_list1+zz_list2)
    times = zz_indices0 + zz_indices1 + zz_indices2
    dgms, generator_list = scListZigzagHomology(times)
    et = time.time()
    zigzag_time = et - st
    print("Cubical zigzag time:", zigzag_time)
    results.append(len(l0[0]))
    results.append(len(l1[0]))
    results.append(len(l2[0]))
    results.append(complex_time)
    results.append(zigzag_time)
    
    st = time.time()
    l0 = []
    l1 = []
    l2 = []
    for img in imageList:    
        sc0,sc1,sc2,sc3 = binaryImageToVietorisRipsComplexList(img)
        l0.append(sc0)
        l1.append(sc1)
        l2.append(sc2)
    et = time.time()
    complex_time = et - st    
    print("Vietoris-Rips complex time:", complex_time)
    results.append(len(l0[0]))
    results.append(len(l1[0]))
    results.append(len(l2[0]))
    results.append(complex_time)    

    return results

In [6]:
def folderListSizeAndTimeComparison (folder, dimensions=[0,1],interval_l=1, gen_l1=1, gen_l2=0,printGenerators=False):   
    
    parent = os.path.dirname(folder)
    name = os.path.basename(folder)    
    outputFolder = os.path.join(parent,"output_"+name)
    
    if not os.path.exists(outputFolder):
        os.makedirs(outputFolder)

    # open the file in the write mode
    file = open(os.path.join(outputFolder,"results.csv"), 'w')

    # create the csv writer
    writer = csv.writer(file)
    
    fields = ["Folder"]
    
    for i in dimensions:
        fields.append("H"+str(i)+"BarNum")
        
    fields = fields +["SmallSC0","SmallSC1","SmallSC2","SmallComplexesTime","SmallZigzagTime"]
    
    fields = fields +["CSCT0","CSCT1","CSCT2","CSCTComplexesTime","CSCTZigzagTime","VRS0","VRS1","VRS2","VRComplexesTime"]

    # write a row to the csv file
    writer.writerow(fields)
    file.flush()
    
    for file_path in natsorted(os.listdir(folder)):
        # check if current file_path is a file
        if ( os.path.isdir(os.path.join(folder, file_path)) and (not ("checkpoints" in file_path))):           
            print("Processing folder",file_path)
            results = imageListSizeAndTimeComparison (os.path.join(folder, file_path),os.path.join(outputFolder,file_path),
                                                     dimensions=dimensions,interval_l=interval_l, gen_l1=gen_l1, gen_l2=gen_l2,
                                                      printGenerators=printGenerators)
            results = [file_path]+results
            writer.writerow(results)
            file.flush()
    # close the file
    file.close()        

In [7]:
folderListSizeAndTimeComparison("dataset", interval_l=20)

Processing folder acA1920-155um__22949301__20200710_094124535
Small complex time: 11.48431944847107
Small zigzag time: 113.19841742515564
Cubical complex time: 5.04277229309082
Cubical zigzag time: 202.39946937561035
Vietoris-Rips complex time: 20.040382385253906
Processing folder acA1920-155um__22949301__20210628_091847394
Small complex time: 12.86234426498413
Small zigzag time: 177.40251231193542
Cubical complex time: 4.903361558914185
Cubical zigzag time: 311.0982024669647
Vietoris-Rips complex time: 26.4152352809906
Processing folder acA1920-155um__22949301__20210628_114754591
Small complex time: 12.937041521072388
Small zigzag time: 149.59269046783447
Cubical complex time: 4.9579761028289795
Cubical zigzag time: 262.8868787288666
Vietoris-Rips complex time: 24.846064567565918
Processing folder casabee1
Small complex time: 12.032095909118652
Small zigzag time: 250.63784003257751
Cubical complex time: 4.561416864395142
Cubical zigzag time: 448.4627459049225
Vietoris-Rips complex tim

<Figure size 640x480 with 0 Axes>

## Read the CSV file to analize the results

The output csv file contains 17 columns:

- **Folder**: name of the video (name of the folder that contains the frame)
- **H0BarNum**:
- **H1BarNum**:
- **SmallSC0**:
- **SmallSC1**:
- **SmallSC2**:
- **SmallComplexesTime**:
- **SmallZigzagTime**:
- **CSCT0**:
- **CSCT1**:
- **CSCT2**:
- **CSCTComplexesTime**:
- **CSCTZigzagTime**:
- **VRS0**:
- **VRS1**:
- **VRS2**:
- **VRComplexesTime**:

In [2]:
import pandas as pd

df = pd.read_csv("output_dataset/results.csv") # Read the results
df

,Folder,H0BarNum,H1BarNum,SmallSC0,SmallSC1,SmallSC2,SmallComplexesTime,SmallZigzagTime,CSCT0,CSCT1,CSCT2,CSCTComplexesTime,CSCTZigzagTime,VRS0,VRS1,VRS2,VRComplexesTime
0,acA1920-155um__22949301__20200710_094124535,18,20,2058,4247,2186,11.484319,113.198417,3015,7130,4112,5.042772,202.399469,2056,5897,4929,20.040382
1,acA1920-155um__22949301__20210628_091847394,19,28,2875,6037,3150,12.862344,177.402512,4147,9901,5742,4.903362,311.098202,2871,8364,7052,26.415235
2,acA1920-155um__22949301__20210628_114754591,22,27,2674,5553,2876,12.937042,149.592690,3900,9243,5340,4.957976,262.886879,2670,7693,6454,24.846065
3,casabee1,26,31,2578,4960,2380,12.032096,250.637840,3924,9006,5080,4.561417,448.462746,2540,6830,5440,23.213639
4,casabee2,13,14,1394,2646,1252,10.595608,117.155608,2144,4896,2752,4.500926,161.531315,1376,3659,2891,12.084023
5,casabee3,13,12,1121,2115,994,10.305248,72.812515,1739,3969,2230,4.349279,110.940546,1115,2962,2338,9.940430
6,casabee4,13,13,1282,2594,1312,11.127613,42.434113,1894,4430,2536,4.771757,63.416304,1268,3544,2918,10.737302
7,casabee5,12,12,1015,1919,904,9.639024,22.801938,1573,3593,2020,4.315916,37.801696,1010,2684,2121,8.798760
8,circles,3,3,472,1028,556,1.961892,14.877367,666,1610,944,0.668227,18.775721,472,1359,1165,3.039694
9,circles_and_squares,2,2,391,935,544,1.532982,17.983871,510,1292,782,0.581486,22.587009,391,1244,1125,2.342135


## Result 1: The new simplicial complex is smaller than the cubical complex and much smaller than the Vietoris-Rips complex.

In [3]:

total_SmallSC0 = df["SmallSC0"].sum()
total_SmallSC1 = df["SmallSC1"].sum()
total_SmallSC2 = df["SmallSC2"].sum()
total_CSCT0 = df["CSCT0"].sum()
total_CSCT1 = df["CSCT1"].sum()
total_CSCT2 = df["CSCT2"].sum()
total_VRS0 = df["VRS0"].sum()
total_VRS1 = df["VRS1"].sum()
total_VRS2 = df["VRS2"].sum()
print("Comparing the size of the new complexes with the cubical complexes:")
print("Ratio SmallSC0/CSCT0:", total_SmallSC0/total_CSCT0)
print("Ratio SmallSC1/CSCT1:", total_SmallSC1/total_CSCT1)
print("Ratio SmallSC2/CSCT2:", total_SmallSC2/total_CSCT2)
print("Comparing the size of the new complexes with the Vietoris-Rips complexes:")
print("Ratio SmallSC0/VRS0:", total_SmallSC0/total_VRS0)
print("Ratio SmallSC1/VRS1:", total_SmallSC1/total_VRS1)
print("Ratio SmallSC2/VRS2:", total_SmallSC2/total_VRS2)

Comparing the size of the new complexes with the cubical complexes:
Ratio SmallSC0/CSCT0: 0.7632585564668348
Ratio SmallSC1/CSCT1: 0.7185532821880812
Ratio SmallSC2/CSCT2: 0.6892090432646975
Comparing the size of the new complexes with the Vietoris-Rips complexes:
Ratio SmallSC0/VRS0: 1.0019954827533277
Ratio SmallSC1/VRS1: 0.7388948636617341
Ratio SmallSC2/VRS2: 0.47387862796833774


## Result 2: The computing time of the new simplicial complex is higher than the cubical complex and lower than the Vietoris-Rips complex

In [4]:
total_SmallComplexesTime = df["SmallComplexesTime"].sum()
total_CSCTComplexesTime = df["CSCTComplexesTime"].sum()
total_VRComplexesTime = df["VRComplexesTime"].sum()
print("Total time to build the new simplicial complexes:", total_SmallComplexesTime)
print("Total time to build the cubical complexes:", total_CSCTComplexesTime)
print("Total time to build the VIetoris-Rips complexes:", total_VRComplexesTime)
print("Ratio Time to build the new simplicial complexes and the cubical complexes:", total_SmallComplexesTime/total_CSCTComplexesTime)
print("Ratio Time to build the new simplicial complexes and the Vietoris-Rips complexes:", total_SmallComplexesTime/total_VRComplexesTime)

Total time to build the new simplicial complexes: 286.6164391040802
Total time to build the cubical complexes: 114.53369855880737
Total time to build the VIetoris-Rips complexes: 644.3541474342346
Ratio Time to build the new simplicial complexes and the cubical complexes: 2.5024638399930557
Ratio Time to build the new simplicial complexes and the Vietoris-Rips complexes: 0.44481197218232144


## Result 3: The total computation time (simplicial complex construction + zigzag persistence) is much better with the new simplicial complex

In [5]:
total_time_small = df["SmallComplexesTime"].sum() + df["SmallZigzagTime"].sum()
total_time_cubical = df["CSCTComplexesTime"].sum() + df["CSCTZigzagTime"].sum()
print("Total time new:", total_time_small)
print("Total time cubical:", total_time_cubical)
print("Ratio", total_time_small/total_time_cubical) # Around 23% of reduction!

Total time new: 9296.929713249207
Total time cubical: 11963.210891485214
Ratio 0.7771266257511412


Note that the zigzag persistence is not computed with the Vietoris-Rips complex, since Dionysus does not allow it. However, given the number of vertices and edges that appear in the Vietoris-Rips complexes (much higher), the computation will be slower than the new approach.

## Result 4: Performance of the method compared to the ground truth

In [6]:
df_ground_truth = pd.read_csv("ground_truth.txt")
df_ground_truth

,folder,H0,H1
0,acA1920-155um__22949301__20200710_094124535,18,21
1,acA1920-155um__22949301__20210628_091847394,19,31
2,acA1920-155um__22949301__20210628_114754591,22,27
3,casabee1,26,30
4,casabee2,13,15
5,casabee3,13,13
6,casabee4,13,13
7,casabee5,12,12
8,circles,3,3
9,circles_and_squares,2,2


False positives are not possible (connected components or holes detected but not appearing in the videos).
So we compute the total real number of H0 and H1 (the ground truth) and also the total number of H0 and H1 detected components.

In [7]:
total_H0_truth = df_ground_truth["H0"].sum()
total_H1_truth = df_ground_truth["H1"].sum()
total_H0_detected = df["H0BarNum"].sum()
total_H1_detected = df["H1BarNum"].sum()
print("Total H0 ground truth:", total_H0_truth)
print("Total H1 ground truth:", total_H1_truth)
print("Total H0 detected:", total_H0_detected)
print("Total H1 detected:", total_H1_detected)
print("Accuracy") # Correct classifications / total classifications
print("Accuracy H0", total_H0_detected/total_H0_truth)
print("Accuracy H1", total_H1_detected/total_H1_truth)

Total H0 ground truth: 191
Total H1 ground truth: 204
Total H0 detected: 189
Total H1 detected: 199
Accuracy
Accuracy H0 0.9895287958115183
Accuracy H1 0.9754901960784313
